In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import dtale

In [2]:
pd.set_option('display.max_columns', None)
%matplotlib inline

In [3]:
def convert_dtype(x):
    if not x:
        return ''
    try:
        return str(x)
    except:
        return ''
def convert_dtypea(x):
    if not x:
        return ''
    try:
        return int(x)
    except:
        return ''
    
smile = pd.read_csv('../data/Smileon.2021.10.17.csv', skiprows=2, header=0, encoding='latin-1', converters={
     'LinkedToCallReportNum':convert_dtype,'ThirdPartyStateProvince':convert_dtype, 'FeedbackFromPhoneWorkerName':convert_dtype,
'SMILE ON 60+ Demographic Information - # of People in Household':convert_dtypea, 'Birth Year':convert_dtypea, 'SMILE ON 60+ Oral Care Encounter - 2393 - Quantity':convert_dtypea,
'SMILE ON 60+ Oral Care Encounter - 2950 - Quantity':convert_dtypea, 'SMILE ON 60+ Oral Care Encounter - 4342 - Quantity':convert_dtypea,
'SMILE ON 60+ Oral Care Encounter - 7140 - Quantity':convert_dtypea, 'SMILE ON 60+ Oral Care Encounter - 7320 - Quantity':convert_dtypea,
'SMILE ON 60+ Oral Care Encounter - Lab Assist narrative':convert_dtype, 'SMILE ON 60+ Case Management Information - What is the outcome for the client?':convert_dtype,
'SMILE ON 60+ Case Management Information - Dental Clinic':convert_dtype})

In [4]:
df = smile

In [6]:
df = df.drop(columns = 'SMILE ON 60+ Base-line Oral and Conditions Questions - Dental Clinic')
df = df.drop(columns = 'SMILE ON 60+ Case Management Information - Dental Clinic')
df = df.drop(columns = 'SMILE ON 60+ Presumptive Eligibility - What is the next step for clients encounter?')
df = df.drop(columns = 'SMILE ON 60+ Oral Care Encounter - Was the enrollee able to be restored to function (can chew) and "social six" esthetics (top front six teeth are present and disease free)')

In [7]:
##used this for general data cleaning
df.columns = [x.replace('SMILE ON 60+ Demographic Information -','') for x in df.columns]
df.columns = [x.replace(' - Quantity','') for x in df.columns]
df.columns = [x.replace('SMILE ON 60+ Screening -','') for x in df.columns]
df.columns = [x.replace('SMILE ON 60+ Base-line Oral and Conditions Questions - ','') for x in df.columns]
df.columns = [x.replace('SMILE ON 60+ Case Management Information - ','') for x in df.columns]
df.columns = [x.replace('(Select all that apply)','') for x in df.columns]
df.columns = [x.replace('SMILE ON 60+ Case Management Information - ','') for x in df.columns]
df.columns = [x.replace('SMILE ON 60+ Demographic Information - ','') for x in df.columns]
df.columns = [x.replace('SMILE ON 60+ Screening -','') for x in df.columns]
df.columns = [x.replace('SMILE ON 60+ Treatment Plan -','') for x in df.columns]
df.columns = [x.replace('SMILE ON 60+ Clinic Information - ','') for x in df.columns]

In [12]:
df.columns[1]

'ReportVersion'

In [8]:
df.drop(df.columns[], axis=1)

,CallReportNum,ReportVersion,LinkedToCallReportNum,CallDateAndTimeStart,CallDateAndTimeEnd,CallLength,CallerNum,PhoneWorkerNum,WasRealCall,WasHangup,WasSexCall,WasWrongNumber,WasPrankCall,WasSilentCall,GeoCode,GeoAssignment,CityName,CountyName,StateProvince,CountryName,PostalCode,CensusDivision,CensusTrack,CensusReportingArea,211Region,PhoneType,ThirdPartyName,ThirdPartyOrganization,ThirdPartyPhoneNumber,ThirdPartyAddress,ThirdPartyCity,ThirdPartyCounty,ThirdPartyStateProvince,ThirdPartyPostalCode,Narrative,VolunteerComments,Feedback,CallersFeedback,TextField2,TextField3,TextField4,TextField5,TextField6,TextField7,TextField8,TextField9,TextField10,EnteredByWorkerNum,EnteredOn,Supervisor,Reviewed,FeedbackStatus,FeedbackFromPhoneWorkerNum,FeedbackFromPhoneWorkerName,ReferralsMade,IPAddress,OrgNum,VolIdentifier,# of People in Household,Birth Year,Other Language,SMILE ON 60+ Notes (Optional) - Notes,SMILE ON 60+ Oral Care Encounter - 1351,SMILE ON 60+ Oral Care Encounter - 1354,SMILE ON 60+ Oral Care Encounter - 2140,SMILE ON 60+ Oral Care Encounter - 2150,SMILE ON 60+ Oral Care Encounter - 2160,SMILE ON 60+ Oral Care Encounter - 2161,SMILE ON 60+ Oral Care Encounter - 220,SMILE ON 60+ Oral Care Encounter - 230,SMILE ON 60+ Oral Care Encounter - 2330,SMILE ON 60+ Oral Care Encounter - 2331,SMILE ON 60+ Oral Care Encounter - 2332,SMILE ON 60+ Oral Care Encounter - 2335,SMILE ON 60+ Oral Care Encounter - 2390,SMILE ON 60+ Oral Care Encounter - 2391,SMILE ON 60+ Oral Care Encounter - 2392,SMILE ON 60+ Oral Care Encounter - 2393,SMILE ON 60+ Oral Care Encounter - 2394,SMILE ON 60+ Oral Care Encounter - 2915,SMILE ON 60+ Oral Care Encounter - 2920,SMILE ON 60+ Oral Care Encounter - 2933,SMILE ON 60+ Oral Care Encounter - 2940,SMILE ON 60+ Oral Care Encounter - 2950,SMILE ON 60+ Oral Care Encounter - 2954,SMILE ON 60+ Oral Care Encounter - 2955,SMILE ON 60+ Oral Care Encounter - 2980,SMILE ON 60+ Oral Care Encounter - 3220,SMILE ON 60+ Oral Care Encounter - 4341,SMILE ON 60+ Oral Care Encounter - 4342,SMILE ON 60+ Oral Care Encounter - 6930,SMILE ON 60+ Oral Care Encounter - 6980,SMILE ON 60+ Oral Care Encounter - 7111,SMILE ON 60+ Oral Care Encounter - 7140,SMILE ON 60+ Oral Care Encounter - 7210,SMILE ON 60+ Oral Care Encounter - 7220,SMILE ON 60+ Oral Care Encounter - 7230,SMILE ON 60+ Oral Care Encounter - 7240,SMILE ON 60+ Oral Care Encounter - 7241,SMILE ON 60+ Oral Care Encounter - 7250,SMILE ON 60+ Oral Care Encounter - 7261,SMILE ON 60+ Oral Care Encounter - 7285,SMILE ON 60+ Oral Care Encounter - 7286,SMILE ON 60+ Oral Care Encounter - 7310,SMILE ON 60+ Oral Care Encounter - 7311,SMILE ON 60+ Oral Care Encounter - 7320,SMILE ON 60+ Oral Care Encounter - 7321,SMILE ON 60+ Oral Care Encounter - 7410,SMILE ON 60+ Oral Care Encounter - 7411,SMILE ON 60+ Oral Care Encounter - 7412,SMILE ON 60+ Oral Care Encounter - 7413,SMILE ON 60+ Oral Care Encounter - 7414,SMILE ON 60+ Oral Care Encounter - 7415,SMILE ON 60+ Oral Care Encounter - 7440,SMILE ON 60+ Oral Care Encounter - 7450,SMILE ON 60+ Oral Care Encounter - 7451,SMILE ON 60+ Oral Care Encounter - 7460,SMILE ON 60+ Oral Care Encounter - 7461,SMILE ON 60+ Oral Care Encounter - 7471,SMILE ON 60+ Oral Care Encounter - 7510,SMILE ON 60+ Oral Care Encounter - 7511,SMILE ON 60+ Oral Care Encounter - 7970 - Mandibular Quantity,SMILE ON 60+ Oral Care Encounter - 7970 - Maxillary Quantity,SMILE ON 60+ Oral Care Encounter - 7971,SMILE ON 60+ Oral Care Encounter - 7972,SMILE ON 60+ Oral Care Encounter - 9110,SMILE ON 60+ Oral Care Encounter - 9951,SMILE ON 60+ Oral Care Encounter - Lab Assist narrative,SMILE ON 60+ Oral Care Encounter - No Code - Denture Step Mandibular Quantity,SMILE ON 60+ Oral Care Encounter - No Code - Denture Step Maxillary Quantity,Last Screening Date,SMILE ON 60+ Verification of Eligibility - Verification Date,Are you limited in what you can eat?,Did the client commit to referral?,Did you help in these other areas?,Do you need help in these other areas?,Have you eve

In [9]:
d = dtale.show(df)
d.open_browser()

2021-11-11 21:00:53,648 - INFO     - NumExpr defaulting to 4 threads.


In [10]:
caller_3638169 = df.loc[df.CallerNum == 3638169]
caller_3638169

,CallReportNum,ReportVersion,LinkedToCallReportNum,CallDateAndTimeStart,CallDateAndTimeEnd,CallLength,CallerNum,PhoneWorkerNum,WasRealCall,WasHangup,WasSexCall,WasWrongNumber,WasPrankCall,WasSilentCall,GeoCode,GeoAssignment,CityName,CountyName,StateProvince,CountryName,PostalCode,CensusDivision,CensusTrack,CensusReportingArea,211Region,PhoneType,ThirdPartyName,ThirdPartyOrganization,ThirdPartyPhoneNumber,ThirdPartyAddress,ThirdPartyCity,ThirdPartyCounty,ThirdPartyStateProvince,ThirdPartyPostalCode,Narrative,VolunteerComments,Feedback,CallersFeedback,TextField2,TextField3,TextField4,TextField5,TextField6,TextField7,TextField8,TextField9,TextField10,EnteredByWorkerNum,EnteredOn,Supervisor,Reviewed,FeedbackStatus,FeedbackFromPhoneWorkerNum,FeedbackFromPhoneWorkerName,ReferralsMade,IPAddress,OrgNum,VolIdentifier,# of People in Household,Birth Year,Other Language,SMILE ON 60+ Notes (Optional) - Notes,SMILE ON 60+ Oral Care Encounter - 1351,SMILE ON 60+ Oral Care Encounter - 1354,SMILE ON 60+ Oral Care Encounter - 2140,SMILE ON 60+ Oral Care Encounter - 2150,SMILE ON 60+ Oral Care Encounter - 2160,SMILE ON 60+ Oral Care Encounter - 2161,SMILE ON 60+ Oral Care Encounter - 220,SMILE ON 60+ Oral Care Encounter - 230,SMILE ON 60+ Oral Care Encounter - 2330,SMILE ON 60+ Oral Care Encounter - 2331,SMILE ON 60+ Oral Care Encounter - 2332,SMILE ON 60+ Oral Care Encounter - 2335,SMILE ON 60+ Oral Care Encounter - 2390,SMILE ON 60+ Oral Care Encounter - 2391,SMILE ON 60+ Oral Care Encounter - 2392,SMILE ON 60+ Oral Care Encounter - 2393,SMILE ON 60+ Oral Care Encounter - 2394,SMILE ON 60+ Oral Care Encounter - 2915,SMILE ON 60+ Oral Care Encounter - 2920,SMILE ON 60+ Oral Care Encounter - 2933,SMILE ON 60+ Oral Care Encounter - 2940,SMILE ON 60+ Oral Care Encounter - 2950,SMILE ON 60+ Oral Care Encounter - 2954,SMILE ON 60+ Oral Care Encounter - 2955,SMILE ON 60+ Oral Care Encounter - 2980,SMILE ON 60+ Oral Care Encounter - 3220,SMILE ON 60+ Oral Care Encounter - 4341,SMILE ON 60+ Oral Care Encounter - 4342,SMILE ON 60+ Oral Care Encounter - 6930,SMILE ON 60+ Oral Care Encounter - 6980,SMILE ON 60+ Oral Care Encounter - 7111,SMILE ON 60+ Oral Care Encounter - 7140,SMILE ON 60+ Oral Care Encounter - 7210,SMILE ON 60+ Oral Care Encounter - 7220,SMILE ON 60+ Oral Care Encounter - 7230,SMILE ON 60+ Oral Care Encounter - 7240,SMILE ON 60+ Oral Care Encounter - 7241,SMILE ON 60+ Oral Care Encounter - 7250,SMILE ON 60+ Oral Care Encounter - 7261,SMILE ON 60+ Oral Care Encounter - 7285,SMILE ON 60+ Oral Care Encounter - 7286,SMILE ON 60+ Oral Care Encounter - 7310,SMILE ON 60+ Oral Care Encounter - 7311,SMILE ON 60+ Oral Care Encounter - 7320,SMILE ON 60+ Oral Care Encounter - 7321,SMILE ON 60+ Oral Care Encounter - 7410,SMILE ON 60+ Oral Care Encounter - 7411,SMILE ON 60+ Oral Care Encounter - 7412,SMILE ON 60+ Oral Care Encounter - 7413,SMILE ON 60+ Oral Care Encounter - 7414,SMILE ON 60+ Oral Care Encounter - 7415,SMILE ON 60+ Oral Care Encounter - 7440,SMILE ON 60+ Oral Care Encounter - 7450,SMILE ON 60+ Oral Care Encounter - 7451,SMILE ON 60+ Oral Care Encounter - 7460,SMILE ON 60+ Oral Care Encounter - 7461,SMILE ON 60+ Oral Care Encounter - 7471,SMILE ON 60+ Oral Care Encounter - 7510,SMILE ON 60+ Oral Care Encounter - 7511,SMILE ON 60+ Oral Care Encounter - 7970 - Mandibular Quantity,SMILE ON 60+ Oral Care Encounter - 7970 - Maxillary Quantity,SMILE ON 60+ Oral Care Encounter - 7971,SMILE ON 60+ Oral Care Encounter - 7972,SMILE ON 60+ Oral Care Encounter - 9110,SMILE ON 60+ Oral Care Encounter - 9951,SMILE ON 60+ Oral Care Encounter - Lab Assist narrative,SMILE ON 60+ Oral Care Encounter - No Code - Denture Step Mandibular Quantity,SMILE ON 60+ Oral Care Encounter - No Code - Denture Step Maxillary Quantity,Last Screening Date,SMILE ON 60+ Verification of Eligibility - Verification Date,Are you limited in what you can eat?,Did the client commit to referral?,Did you help in these other areas?,Do you need help in these other areas?,Have you eve

In [11]:
df[' # of Upper Natural Teeth'].value_counts()

12.0    633
14.0    616
13.0    515
11.0    501
10.0    491
9.0     469
8.0     432
7.0     374
6.0     317
5.0     242
15.0    184
4.0     183
16.0    115
3.0     110
2.0      89
1.0      81
Name:  # of Upper Natural Teeth, dtype: int64

In [ ]:
print(df.columns.values)